In [2]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =====

## 3. Customize and Extend Merlin Models

Merlin Models provides common and state-of-the-art RecSys architectures in a high-level API as well as all the required low-level building blocks (e.g., input blocks, MLP layers, prediction tasks, loss functions, etc.) for you to create your own architecture. 

In this lab, we define DLRM model architecture from scratch and customize it with Merlin Models.

**Learning Objectives**

- Understand the building blocks of Merlin Models
- Define DLRM model architecture with low-level api
- Customize DLRM model with Merlin Models: 
    - Add cross-product transformation (see [Wide & Deep](https://arxiv.org/abs/1606.07792) paper) to the DLRM model.
    - Replace the pairwise interaction layer of DLRM by a cross network (see [DCN-v2](https://arxiv.org/abs/2008.13535) paper).   

**Import Required Libraries**

In [3]:
import os

import glob
import cudf 
import pandas as pd
import numpy as np
import nvtabular as nvt
from nvtabular.ops import *
import gc

from merlin.schema.tags import Tags
import merlin.models.tf as mm
from merlin.io.dataset import Dataset

import tensorflow as tf

2022-09-09 15:56:58.965415: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-09 15:57:01.655431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16255 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:8a:00.0, compute capability: 7.0


In [4]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

Define data paths.

In [5]:
data_path = '/workspace/data/ecom/'
output_path = os.path.join(data_path,'processed_nvt')

Read processed parquet files as Dataset objects.

In [6]:
train = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

# define schema object
schema = train.schema.without(['event_time_ts', 'user_id_raw', 'product_id_raw'])

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [7]:
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'target'

### 3.1. Introduction to Merlin Models' core building blocks

Let's explain the functions and blocks that we use to build our DLRM model using Merlin Models' blocks and methods:

The `Block` is the core abstraction in Merlin Models and is the class from which all blocks inherit. The class extends the `tf.keras.layers.Layer` base class and implements a number of properties that simplify the creation of custom blocks and models. These properties include the Schema object for determining the embedding dimensions, input shapes, and output shapes.

**Features Blocks** <br>

`Embeddings:` Creates a ParallelBlock with an EmbeddingTable for each categorical feature in the schema. <br>
`ContinuousFeatures:` Input block for continuous features.

**Connects Methods** <br>
The base class `Block` implements different connects methods that control how to link a given block to other blocks. In this lab, we use the following methods:

- `connect:` Connect the block to other blocks sequentially. The output is a tensor returned by the last block.
- `connect_with_shortcut:` Connect the block to other blocks sequentially and apply a skip connection with the block's output.

To learn more about other `connect` methods you can visit [here](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/core/base.py).

### 3.2. Build a DLRM model using Merlin Models low-level API

In this lab, we follow the steps below build our DLRM model from scratch:

- build bottom MLP block that processes numerical features
- build embeddings layer for categorical features
- combine the output of continuous and categorical blocks into one dictionary
- apply a dot product between all continuous and categorical features to learn pairwise interactions
- concat the resulting pairwise interaction with the deep representation of continuous features (skip-connection)
- apply top MLP block with a series of dense layers to the concatenated tensor.

<img src="./images/DLRM.png" width="400" height="200">

We can feed an example batch through Merlin Models layers and get the results. So, let's  convert the first five rows of the valid dataset to a batch of input tensors, so that we can check out the outputs from each block below.

In [8]:
batch = mm.sample_batch(valid, batch_size=5, shuffle=False, include_targets=False)
print(batch)

{'user_id': <tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[184309],
       [184309],
       [184309],
       [184309],
       [184309]], dtype=int32)>, 'ts_weekday': <tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[4],
       [7],
       [7],
       [3],
       [3]], dtype=int32)>, 'ts_hour': <tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[19],
       [ 6],
       [ 6],
       [15],
       [15]], dtype=int32)>, 'product_id': <tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[3372],
       [1822],
       [ 460],
       [ 183],
       [2230]], dtype=int32)>, 'cat_0': <tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[1],
       [2],
       [1],
       [1],
       [1]], dtype=int32)>, 'cat_1': <tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[1],
       [3],
       [1],
       [1],
       [1]], dtype=int32)>, 'cat_2': <tf.Tensor: shape=(5, 1), dtype=int32, numpy=
array([[1],
       [0],
       [1],
       [1],
       [1]], dtype=int32)>, 'brand': <tf.Tensor: sh

We define the continuous layer based on the schema.

In [9]:
continuous_block = mm.ContinuousFeatures.from_schema(schema, tags=Tags.CONTINUOUS)
continuous_block(batch)

{'price': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[1.0036721 ],
        [0.86666507],
        [1.2039791 ],
        [0.64332473],
        [1.0186374 ]], dtype=float32)>,
 'relative_price': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[-0.0670252 ],
        [-0.03655583],
        [ 0.08952092],
        [-0.0848495 ],
        [-0.0449205 ]], dtype=float32)>,
 'TE_user_id_target': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[0.657176  ],
        [0.43412507],
        [1.1355263 ],
        [1.1355263 ],
        [0.43412507]], dtype=float32)>,
 'TE_brand_target': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[0.36400566],
        [0.42630896],
        [0.27878827],
        [0.52409333],
        [0.36629993]], dtype=float32)>,
 'TE_cat_1_target': <tf.Tensor: shape=(5, 1), dtype=float32, numpy=
 array([[0.5736734 ],
        [0.332066  ],
        [0.57252383],
        [0.57252383],
        [0.57481176]], dtype=float32)>,
 'TE_cat_2_target': <tf.

We connect the continuous block to an `MLPBlock` to project them into the same dimensionality as the embedding width of categorical features.

In [10]:
bottom_block = continuous_block.connect(mm.MLPBlock([64]))
bottom_block(batch)

<tf.Tensor: shape=(5, 64), dtype=float32, numpy=
array([[0.24168067, 0.        , 0.        , 0.        , 0.        ,
        0.12457348, 0.06815903, 0.        , 0.33162716, 0.        ,
        0.33803773, 0.3546738 , 0.18753652, 0.23609668, 0.        ,
        0.        , 0.12242705, 0.        , 0.2776879 , 0.        ,
        0.11242545, 0.41953292, 0.        , 0.        , 0.01660733,
        0.11212933, 0.17494613, 0.2146942 , 0.43573064, 0.01884168,
        0.03417264, 0.11854244, 0.6272503 , 0.        , 0.07257415,
        0.        , 0.2819846 , 0.20729566, 0.        , 0.5770575 ,
        0.        , 0.04364566, 0.00127549, 0.        , 0.        ,
        0.        , 0.        , 0.1251218 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.35589513, 0.        , 0.        , 0.11736025, 0.        ,
        0.16131279, 0.        , 0.23884481, 0.03296663],
       [0.        , 0.24042037, 0.12846799, 0.19597757, 0.        ,
        0.

We define the categorical embedding block based on the schema. We display the output tensor of the categorical embedding block using the data from the first batch. We can see the embeddings tensors of categorical features with a default dimension of 64.

In [11]:
embeddings_block = mm.Embeddings(
    schema.select_by_tag(Tags.CATEGORICAL),
    dim = 64
)
embeddings_block(batch)

{'user_id': <tf.Tensor: shape=(5, 64), dtype=float32, numpy=
 array([[-0.0030684 ,  0.04588665,  0.02369252, -0.04817641, -0.02404811,
         -0.03262407, -0.01784763,  0.02128987,  0.03647913, -0.01208849,
          0.02840331,  0.02881254, -0.0313934 , -0.02020936, -0.0109046 ,
          0.0033321 ,  0.01262108,  0.02360431,  0.01518716,  0.03210591,
         -0.01987057,  0.00957217, -0.01579005, -0.03097498, -0.04053762,
         -0.00797368, -0.03997766, -0.03261774, -0.01614944, -0.04255729,
          0.01050658,  0.02750127,  0.03751229,  0.02619976, -0.02484912,
         -0.02298819,  0.03241647, -0.02462848,  0.02332423,  0.00559973,
         -0.02477183,  0.00516224, -0.01597665,  0.01938793, -0.03212832,
         -0.0299018 , -0.01360755,  0.0384554 ,  0.03403345,  0.00648155,
          0.0200499 ,  0.04552804,  0.01207328,  0.03589824,  0.01733079,
          0.01685769,  0.00346895, -0.03348974, -0.00527056, -0.0014535 ,
          0.04884484,  0.04587479,  0.00918368,  0.

Let's combine the output of continuous and categorical blocks into one dictionary using a `ParallelBlock` instance.

In [12]:
dlrm_input_block = mm.ParallelBlock(
    {"embeddings": embeddings_block, "bottom_block": bottom_block}
)

By looking at the output, we can see that the ParallelBlock class applies embedding and continuous blocks, in parallel, to the same input batch. Additionally, it merges the resulting tensors into one dictionary.

In [13]:
print("Output shapes of DLRM input block:")
for key, val in dlrm_input_block(batch).items():
    print("\t%s : %s" % (key, val.shape))

Output shapes of DLRM input block:
	user_id : (5, 64)
	ts_weekday : (5, 64)
	ts_hour : (5, 64)
	product_id : (5, 64)
	cat_0 : (5, 64)
	cat_1 : (5, 64)
	cat_2 : (5, 64)
	brand : (5, 64)
	bottom_block : (5, 64)


**Define the interaction block**

Now that we have a vector representation of each input feature, we will create the DLRM interaction block. It consists of three operations:

- Apply a dot product between all continuous and categorical features to learn pairwise interactions
- Concat the resulting pairwise interaction with the deep representation of continuous features (skip-connection)
- Apply an `MLPBlock` with a series of dense layers to the concatenated tensor.

First, we use the `connect_with_shortcut` method to create first two operations of the DLRM interaction block:

In [14]:
from merlin.models.tf.blocks.dlrm import DotProductInteractionBlock

dlrm_interaction = dlrm_input_block.connect_with_shortcut(
    DotProductInteractionBlock(), shortcut_filter=mm.Filter("bottom_block"), aggregation="concat"
)

The `Filter` operation allows us to select the deep_continuous tensor from the dlrm_input_block outputs.

The following diagram provides a visualization of the operations that we constructed in the dlrm_interaction object.

<img src="./images/residual_interaction.png" width="300" height="200">

Print the tensor outputs from `dlrm_interaction` block.

In [15]:
dlrm_interaction(batch)

<tf.Tensor: shape=(5, 100), dtype=float32, numpy=
array([[ 2.41680667e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.24573484e-01,
         6.81590289e-02,  0.00000000e+00,  3.31627160e-01,
         0.00000000e+00,  3.38037729e-01,  3.54673803e-01,
         1.87536523e-01,  2.36096680e-01,  0.00000000e+00,
         0.00000000e+00,  1.22427046e-01,  0.00000000e+00,
         2.77687907e-01,  0.00000000e+00,  1.12425454e-01,
         4.19532925e-01,  0.00000000e+00,  0.00000000e+00,
         1.66073292e-02,  1.12129331e-01,  1.74946129e-01,
         2.14694202e-01,  4.35730636e-01,  1.88416839e-02,
         3.41726393e-02,  1.18542440e-01,  6.27250314e-01,
         0.00000000e+00,  7.25741461e-02,  0.00000000e+00,
         2.81984597e-01,  2.07295656e-01,  0.00000000e+00,
         5.77057481e-01,  0.00000000e+00,  4.36456576e-02,
         1.27549469e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.25121802e-01,
      

In [16]:
top_mlp = mm.MLPBlock([128, 64, 32])
dlrm_body = dlrm_interaction.connect(top_mlp)

**Define the Prediction block**

At this stage, we have created the DLRM block that accepts a dictionary of categorical and continuous tensors as input. The output of this block is the interaction representation vector of shape 32. The next step is to use this hidden representation to conduct a given prediction task.

We use the BinaryClassificationTask class and evaluate the performances using the AUC metric.

In [17]:
binary_task = mm.BinaryClassificationTask(schema)

We connect the deep DLRM interaction layer to the binary task head and the method automatically generates the Model class for us. We note that the Model class inherits from `tf.keras.Model` class.

In [18]:
model = mm.Model(dlrm_body, binary_task)

In [19]:
%%time 
model.compile(optimizer='adam', run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

Epoch 1/2
735/735 [==============================] - 21s 17ms/step - loss: 0.5757 - auc: 0.7672 - regularization_loss: 0.0000e+00 - val_loss: 0.7220 - val_auc: 0.6388 - val_regularization_loss: 0.0000e+00
Epoch 2/2
735/735 [==============================] - 11s 14ms/step - loss: 0.4307 - auc: 0.8818 - regularization_loss: 0.0000e+00 - val_loss: 0.8016 - val_auc: 0.6311 - val_regularization_loss: 0.0000e+00
CPU times: user 47 s, sys: 7.48 s, total: 54.5 s
Wall time: 34.1 s


### 3.3. Customize DLRM architecture: Add Cross-Product features to DLRM Model

We can synthetically form new features by multiplying (crossing) two or more sparse features. Crossing combinations of features can provide predictive abilities beyond what those features provide individually (see ref [website](https://developers.google.com/machine-learning/crash-course/feature-crosses/video-lecture#:~:text=A%20feature%20cross%20is%20a,an%20understanding%20of%20feature%20crosses)). In particular, cross-product feature transformations help the model memorize the niche/rare interactions seen in the user’s past history. Feature crossing has been used as an efficient technique in well-known DL architectures such as [Wide & Deep](https://arxiv.org/abs/1606.07792) in the Wide part. The `HashedCross` class in Merlin Models allows us to perform cross-product transformations between two or multiple sparse categorical features. By using the <i>hashing trick</i>, we can control the dimension of the resulting crossed features. Conceptually, the transformation can be thought of as: `hash(concatenation of features) % num_bins`. 

The purpose of this use case is to illustrate how two categorical variables can be combined to generate one cross-product feature. In more general situations, where you want to create multiple crossed features from a list of categorical variables, you can use the `HashCrossAll` class.

Instead of re-building main blocks from scratch, at this step, we can use `DLRMBlock` instead to define the same DLRM architecture defined above.

In [20]:
dlrm_body = mm.DLRMBlock(schema, embedding_dim=64, bottom_block=mm.MLPBlock([128,64]), top_block= mm.MLPBlock([128, 64, 32]))

In [21]:
dlrm_body(batch)

<tf.Tensor: shape=(5, 32), dtype=float32, numpy=
array([[0.        , 0.02928996, 0.0473752 , 0.06288711, 0.        ,
        0.0638621 , 0.        , 0.02099078, 0.        , 0.00656151,
        0.01406717, 0.00405715, 0.02685894, 0.        , 0.08501423,
        0.        , 0.        , 0.        , 0.04040261, 0.        ,
        0.06904767, 0.00144228, 0.        , 0.        , 0.00687732,
        0.        , 0.024181  , 0.03958534, 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.02827137, 0.04741701, 0.        ,
        0.01240498, 0.00037761, 0.06874807, 0.04542917, 0.        ,
        0.0212361 , 0.        , 0.05115933, 0.        , 0.14823578,
        0.        , 0.        , 0.        , 0.07170902, 0.01802336,
        0.05887214, 0.00694745, 0.02424705, 0.        , 0.        ,
        0.04553917, 0.03927574, 0.        , 0.08305362, 0.03873212,
        0.        , 0.        ],
       [0.        , 0.02731217, 0.05164035, 0.09959982, 0.        ,
 

Define `cross_features` transformation block.

In [22]:
cross_schema = schema.select_by_name(names=["cat_0", "cat_1"])
cross_features = mm.HashedCross(cross_schema, num_bins=100, output_mode="one_hot")

Print the tensor outputs from `cross_features` block.

In [23]:
cross_features(batch)

{'cross_cat_0_cat_1': <tf.Tensor: shape=(5, 100), dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],


To learn more about the `HashedCross` class arguments (e.g., num_bins, output_mode, etc) you can visit [here](https://github.com/NVIDIA-Merlin/models/blob/main/merlin/models/tf/core/transformations.py#L825-L846).

For large cardinality, it is important to set `sparse=True` for `HashedCross`, `HashedCrossAll` and `CategoryEncoding`, as it makes output sparse (otherwise this might cause OOM). But for this example is fine to keep it as the default `sparse=False`.

We add another feature interaction representation based on the weighted sum of feature crosses, thefore, we connect the `HashedCross` transformation block with a simple single-neuron linear MLP architecture.

In [24]:
# wide part: 

wide_body = cross_features.connect(
    mm.MLPBlock([1], no_activation_last_layer=True), block_name='cross_model'
)

In [25]:
wide_body(batch)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.15635493],
       [ 0.2237919 ],
       [-0.15635493],
       [-0.15635493],
       [-0.15635493]], dtype=float32)>

Concat `wide_body` layer to `dlrm_body` layer using `ParallelBlock`.

In [26]:
# wide-and-dlrm
wide_and_dlrm = mm.ParallelBlock({'wide':wide_body, "dlrm": dlrm_body}, aggregation="concat")
wide_and_dlrm(batch)

<tf.Tensor: shape=(5, 33), dtype=float32, numpy=
array([[ 0.        ,  0.02928996,  0.0473752 ,  0.06288711,  0.        ,
         0.0638621 ,  0.        ,  0.02099078,  0.        ,  0.00656151,
         0.01406717,  0.00405715,  0.02685894,  0.        ,  0.08501423,
         0.        ,  0.        ,  0.        ,  0.04040261,  0.        ,
         0.06904767,  0.00144228,  0.        ,  0.        ,  0.00687732,
         0.        ,  0.024181  ,  0.03958534,  0.        ,  0.        ,
         0.        ,  0.        , -0.15635493],
       [ 0.        ,  0.        ,  0.02827137,  0.04741701,  0.        ,
         0.01240498,  0.00037761,  0.06874807,  0.04542917,  0.        ,
         0.0212361 ,  0.        ,  0.05115933,  0.        ,  0.14823578,
         0.        ,  0.        ,  0.        ,  0.07170902,  0.01802336,
         0.05887214,  0.00694745,  0.02424705,  0.        ,  0.        ,
         0.04553917,  0.03927574,  0.        ,  0.08305362,  0.03873212,
         0.        ,  0.   

In [25]:
binary_task = mm.BinaryClassificationTask(schema)
model = mm.Model(wide_and_dlrm, binary_task)

In [26]:
%%time 
model.compile(optimizer='adam', run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

Epoch 1/2
735/735 [==============================] - 20s 20ms/step - loss: 0.5723 - auc_1: 0.7713 - regularization_loss: 0.0000e+00 - val_loss: 0.7445 - val_auc_1: 0.6391 - val_regularization_loss: 0.0000e+00
Epoch 2/2
735/735 [==============================] - 14s 18ms/step - loss: 0.4417 - auc_1: 0.8748 - regularization_loss: 0.0000e+00 - val_loss: 0.8070 - val_auc_1: 0.6350 - val_regularization_loss: 0.0000e+00
CPU times: user 53 s, sys: 7.1 s, total: 1min
Wall time: 35.1 s


### 3.4. Replace DotProductInteractionBlock with CrossBlock

In this section, we will replace DotProductInteractionBlock layer with `CrossBlock` that is used in [DCN-v2](https://arxiv.org/pdf/2008.13535.pdf) architecture. CrossBlock uses `Cross` layer which creates interactions of all input features. When used inside `CrossBlock`, stacked `Cross` layers can be used to do high-order features interaction.

We will keep bottom layer same as we did above.

In [27]:
continuous_block = mm.ContinuousFeatures.from_schema(schema, tags=Tags.CONTINUOUS)
bottom_block = continuous_block.connect(mm.MLPBlock([128,64]))

In [28]:
embeddings_block = mm.Embeddings(
    schema.select_by_tag(Tags.CATEGORICAL),
    dim = 64
)
embeddings = embeddings_block(batch)
embeddings.keys(), embeddings["user_id"].shape

(dict_keys(['user_id', 'ts_weekday', 'ts_hour', 'product_id', 'cat_0', 'cat_1', 'cat_2', 'brand']),
 TensorShape([5, 64]))

In [29]:
dlrm_input_block = mm.ParallelBlock(
    {"embeddings": embeddings_block, "bottom_block": bottom_block},
    aggregation="concat"
)

`CrossBlock` block provides a way to create high-order feature interactions by a number of stacked Cross Layers. We set the depth, which is the number of cross-layers to be stacked, to 2.

In [30]:
#stacked
cross_inter_body = dlrm_input_block.connect(mm.CrossBlock(2))
cross_inter_body(batch)

<tf.Tensor: shape=(5, 576), dtype=float32, numpy=
array([[ 0.01346518,  0.01996166,  0.10235769, ..., -0.01322006,
        -0.00873787, -0.03701539],
       [ 0.13175648,  0.03475271,  0.        , ..., -0.01474485,
        -0.00794951, -0.03636587],
       [ 0.        ,  0.05581063,  0.10123038, ..., -0.0129318 ,
        -0.00880974, -0.03923511],
       [ 0.        ,  0.04674373,  0.12530845, ..., -0.0126127 ,
        -0.00837193, -0.04131868],
       [ 0.01239133,  0.02843624,  0.11145297, ..., -0.01299973,
        -0.00858843, -0.03969663]], dtype=float32)>

Concat `cross_inter_body` layer to bottom block using ParallelBlock.

In [31]:
dlrm_interaction = mm.ParallelBlock(
    {"cross_inter_body": cross_inter_body, "bottom_block": bottom_block},
    aggregation="concat"
)                                                

Then, we project the learned interaction using a series of dense layers, this defines the top block.

In [32]:
deep_dlrm_interaction = dlrm_interaction.connect(mm.MLPBlock([128, 64, 32]))

In [33]:
binary_task = mm.BinaryClassificationTask(schema)

We connect the deep DLRM interaction layer to the binary task head, and automatically generate the Model class.

In [34]:
model = mm.Model(deep_dlrm_interaction, binary_task)

In [35]:
%%time 
model.compile(optimizer='adam', run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

Epoch 1/2
735/735 [==============================] - 18s 19ms/step - loss: 0.5677 - auc_2: 0.7759 - regularization_loss: 0.0000e+00 - val_loss: 0.7465 - val_auc_2: 0.6404 - val_regularization_loss: 0.0000e+00
Epoch 2/2
735/735 [==============================] - 12s 16ms/step - loss: 0.4573 - auc_2: 0.8644 - regularization_loss: 0.0000e+00 - val_loss: 0.8592 - val_auc_2: 0.6394 - val_regularization_loss: 0.0000e+00
CPU times: user 46.4 s, sys: 6.91 s, total: 53.3 s
Wall time: 32.2 s


### Summary 

In this hands-on lab we learned how to

- use a subset of pre-existing blocks to create a DLRM model
- add cross-product transformation block to the DLRM Model
- replace DotProductInteractionBlock with CrossBlock

Please execute the cell below to shut down the kernel before moving on to the next notebook, `04-Building-multi-stage-RecSys-with-Merlin-Systems`.

In [36]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}